###### DBSCAN clustering algorithm
<img src="https://drive.google.com/uc?export=view&id=1Bdj6aueXeGtPNur7dgJdn4drJwKaPjzf" width="400" height="200">

In [2]:
import os
import time

# Spark
import pyspark.sql.functions as f
import pyspark.sql.types as T

# Data transformation
import pandas as pd
from pyspark.sql import Window
from pyspark.sql import DataFrame
from functools import reduce 
from pyspark.sql.functions import udf

# Feature engineering
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline

# Linear algebra and visualization tools
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm

# Machine learning
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors
from sklearn import metrics
from geopy.distance import great_circle
from shapely.geometry import MultiPoint

# SparkMLib
from pyspark.ml.clustering import KMeans
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler

# Interactive maps
import folium

In [3]:
def get_pcd(addr):
  try: pcd = str(addr).split(',')[-1].strip()
  except: pcd = None
  return pcd

get_pcd_udf = f.udf(lambda x: get_pcd(x), T.StringType())

###### read the data

In [5]:
dbutils.fs.ls('mnt/data-one-off-legacy/data_requests/')

Out[4]: [FileInfo(path='dbfs:/mnt/data-one-off-legacy/data_requests/Donations.xlsx', name='Donations.xlsx', size=52549)]

In [6]:
df_geocode_pcd = spark.table('prod_dataoneoff_others.parsed_postcode_gps')

pdf_carehomes = pd.read_excel('/dbfs/mnt/data-one-off-legacy/data_requests/Donations.xlsx')
pdf_gp = pd.read_excel('/dbfs/mnt/data-one-off-legacy/data_requests/Donations.xlsx', 'GP Practices ')
df_carehomes = sqlContext.createDataFrame(pdf_carehomes)
df_carehomes = df_carehomes.withColumn('pcd', get_pcd_udf(f.col('Address ')))
df_gp = sqlContext.createDataFrame(pdf_gp)

##### geocode the data using pcd lkp

In [8]:
df_gp_locations = df_gp.join(df_geocode_pcd,  f.lower(df_geocode_pcd.Postcode) == f.regexp_replace(f.lower(df_gp.Postcode), ' ', '')).select(df_gp['*'], 'lat', 'lng')

In [9]:
display(df_gp_locations)

Locality,Neighbourhood,Practice_Name,Adress1,Address2,Address3,Address4,Postcode,lat,lng
South,Wythenshawe & Northenden,Northern Moor Medical Practice,216 Wythenshawe Road,Northern Moor,Manchester,null,M23 0PH,53.408047946114,-2.285108643386
Central,"Hulme, Moss Side & Rusholme",The Whitswood Practice,Alexandra Park Health Centre,2 Whitswood Close,Moss Side,Manchester,M16 7AP,53.455776501226,-2.253502382674
South,Withington & Fallowfield,Al-Shifa Medical Centre,Al-Shifa Medical Centre,6 Copson Street,Withington,Manchester,M20 3HE,53.433712955677,-2.229694627192
Central,Ardwick & Longsight,Drs Ngan & Chan,The Vallance Centre,Brunswick Street,Manchester,null,M13 9UJ,53.469812099046,-2.224270658805
Central,Ardwick & Longsight,Dr Cunningham & Partners,The Vallance Centre,Brunswick Street,Manchester,null,M13 9UJ,53.469812099046,-2.224270658805
Central,Ardwick & Longsight,"Drs Chiu, Koh & Gan",The Vallance Centre,Brunswick Street,Manchester,null,M13 9UJ,53.469812099046,-2.224270658805
Central,"Hulme, Moss Side & Rusholme",Wilmslow Road Medical Centre,Wilmslow Road Medical Centre,156a Wilmslow Road,Rusholme,Manchester,M14 5LQ,53.452161482765,-2.222792200422
Central,"Chorlton, Whalley Range & Fallowfield",Ashville Surgery,Ashville Surgery,171 Upper Chorlton Road,Whalley Range,Manchester,M16 9RT,53.45251785859,-2.268044376555
North,"Ancoats, Clayton & Bradford",New Islington Medical Centre,Ancoats Primary Care Centre,Old Mill Street,Manchester,Lancashire,M4 6EE,53.482845160638,-2.219547505109
North,"Ancoats, Clayton & Bradford",Urban Village Medical Practice,Ancoats Primary Care Centre,Old Mill Street,Ancoats,Manchester,M4 6EE,53.482845160638,-2.219547505109


##### GPs distribution
###### The graph below shows us the geographic distribution of GP facilities according to their coordinates

In [11]:
# extract GPs coordinates from dataframe
gp_coordinates = (df_gp_locations
                        .select('lat', 'lng')
                        .collect())
lat, lng = zip(*gp_coordinates)

In [12]:
fig, ax = plt.subplots(figsize=[20, 12])
gp_scatter = ax.scatter(lng, lat, c='#99cc99', edgecolor='None', alpha=0.9, s=120)
ax.set_title('GPs', fontsize = 30)
ax.set_xlabel('Longitude', fontsize=20)
ax.set_ylabel('Latitude', fontsize =20)
ax.legend([gp_scatter], ['GPs'], loc='upper right', fontsize = 20)
display(fig)

##### DBSCAN vs K-means
###### Density-Based Spatial Clustering and Application with Noise, is a density-based clusering algorithm aimed for spatial data

<img src="https://drive.google.com/uc?export=view&id=102JZEkPMIscqwfNhyukczXvqFRqB2gK-" width="200" height="100">

###### K-means is a similar clustering algorithm but doesn't care about how much dense the data is present

<img src="https://drive.google.com/uc?export=view&id=1FlX0AHcinDJ6TsQ50jCbI8OweTH9Hulj" width="200" height="100">

In [14]:
# convert Spark DataFrames to Pandas DataFrames
pd_gp_locations = df_gp_locations.toPandas()

/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "

In [15]:
# define the number of kilometers in one radiation
# which will be used to convert esp from km to radiation
kms_per_rad = 6371.0088

In [16]:
# define a function to calculate the geographic coordinate 
# centroid of a cluster of geographic points
# it will be used later to calculate the centroids of DBSCAN cluster
# because Scikit-learn DBSCAN cluster class does not come with centroid attribute.
def get_centroid(cluster):
  """calculate the centroid of a cluster of geographic coordinate points
  Args:
    cluster coordinates, nx2 array-like (array, list of lists, etc) 
    n is the number of points(latitude, longitude)in the cluster.
  Return:
    geometry centroid of the cluster
    
  """
  cluster_ary = np.asarray(cluster)
  centroid = cluster_ary.mean(axis = 0)
  return centroid

# testing get_centroid function
test_cluster= [[ 43.70487299, -79.57753802], 
               [ 43.71138367, -79.56524418],
               [ 43.72616079, -79.57319998],
               [ 43.73547907, -79.56258364],
               [ 43.72070325, -79.57202018],
               [ 43.73126031, -79.5598719 ]]
test_centroid = get_centroid(test_cluster)
print(test_centroid)

[ 43.72164335 -79.56840965]

###### Parameters

In [18]:
# Epsilon defines the radius of neighborhood around a point x. We need to convert eps to radians for use by haversine
epsilon = 1.39/kms_per_rad

# minPts: it is the minimum number of neighbors within eps radius
minPts = 1

# Extract intersection coordinates (latitude, longitude)
coords = pd_gp_locations.as_matrix(columns = ['lat', 'lng'])

/local_disk0/tmp/1589441408193-0/PythonShell.py:8: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
 import signal

###### Algorithm

In [20]:
start_time = time.time()
dbsc = (DBSCAN(eps=epsilon, min_samples=minPts, algorithm='ball_tree', metric='haversine')
        .fit(np.radians(coords)))
cluster_labels = dbsc.labels_

In [21]:
# get the number of clusters
num_clusters = len(set(dbsc.labels_))

In [22]:
# print the outcome
message = 'Clustered {:,} points down to {:,} clusters, for {:.1f}% compression in {:,.2f} seconds'
print(message.format(len(pd_gp_locations), num_clusters, 100*(1 - float(num_clusters) / len(pd_gp_locations)), time.time()-start_time))
print('Silhouette coefficient: {:0.03f}'.format(metrics.silhouette_score(coords, cluster_labels)))

# turn the clusters into a pandas series,where each element is a cluster of points
dbsc_clusters = pd.Series([coords[cluster_labels==n] for n in range(num_clusters)])

Clustered 83 points down to 6 clusters, for 92.8% compression in 0.03 seconds
Silhouette coefficient: 0.181

In [23]:
# get centroid of each cluster
centroids = dbsc_clusters.map(get_centroid)
# unzip the list of centroid points (lat, lon) tuples into separate lat and lon lists
cent_lats, cent_lons = zip(*centroids)
# from these lats/lons create a new df of one representative point for eac cluster
centroids_pd = pd.DataFrame({'longitude':cent_lons, 'latitude':cent_lats})

##### Visual

In [25]:
pd_cluster_labels = pd.DataFrame(cluster_labels, columns=['cluster_labels'])
pd_output = pd.concat([pd_gp_locations, pd_cluster_labels], axis = 1, sort = False)
pd_output.head()

,Locality,Neighbourhood,Practice_Name,Adress1,Address2,Address3,Address4,Postcode,lat,lng,cluster_labels
0,South,Wythenshawe & Northenden,Northern Moor Medical Practice,216 Wythenshawe Road,Northern Moor,Manchester,None,M23 0PH,53.408048,-2.285109,0
1,Central,"Hulme, Moss Side & Rusholme",The Whitswood Practice,Alexandra Park Health Centre,2 Whitswood Close,Moss Side,Manchester,M16 7AP,53.455777,-2.253502,1
2,South,Withington & Fallowfield,Al-Shifa Medical Centre,Al-Shifa Medical Centre,6 Copson Street,Withington,Manchester,M20 3HE,53.433713,-2.229695,1
3,Central,Ardwick & Longsight,Drs Ngan & Chan,The Vallance Centre,Brunswick Street,Manchester,None,M13 9UJ,53.469812,-2.224271,1
4,Central,Ardwick & Longsight,Dr Cunningham & Partners,The Vallance Centre,Brunswick Street,Manchester,None,M13 9UJ,53.469812,-2.224271,1


In [26]:
fig, ax = plt.subplots()
cmap = plt.cm.get_cmap('jet')
for i, cluster in pd_output.groupby('cluster_labels'):
    _ = ax.scatter(cluster['lng'], cluster['lat'], label=i)
# centroid_scatter = ax.scatter(centroids_pd['longitude'], centroids_pd['latitude'], marker='x', linewidths=2, c='k', s=50)
ax.set_title('DBSCAN Clustering')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.legend()
ax.grid(True)

display()

In [27]:
# Plot the faciity clusters and cluster centroid
fig, ax = plt.subplots(figsize=[20, 12])
gp_scatter = ax.scatter(pd_gp_locations['lng'], pd_gp_locations['lat'], c=cluster_labels, cmap = cm.Dark2, edgecolor='None', alpha=0.7, s=120)
centroid_scatter = ax.scatter(centroids_pd['longitude'], centroids_pd['latitude'], marker='x', linewidths=2, c='k', s=50)
# ax.set_title('GP Clusters & Centroid', fontsize = 30)
ax.set_xlabel('Longitude', fontsize=24)
ax.set_ylabel('Latitude', fontsize = 24)
# ax.legend([gp_scatter, centroid_scatter], ['GPs', 'GP Cluster Centroid'], loc='upper right', fontsize = 20)
# ax.grid(True)

display(fig)

In [28]:
# Determine min and max lat/lng
max_lat = df_gp_locations.agg(f.max(df_gp_locations['lat'])).collect()
min_lat = df_gp_locations.agg(f.min(df_gp_locations['lat'])).collect()
max_lng = df_gp_locations.agg(f.max(df_gp_locations['lng'])).collect()
min_lng = df_gp_locations.agg(f.min(df_gp_locations['lng'])).collect()

In [29]:
# Create tuple of coordinates - min_lng, max_lng, min_lat, max_lat. This is used to set x & y axis (xlim, ylim)
BBox = (-2.312114869047, -2.161372101408, 53.370738221737, 53.534331573503)

In [30]:
# Display tuple
print(BBox)

(-2.312114869047, -2.161372101408, 53.370738221737, 53.534331573503)

In [31]:
# Guide to background image to scatter plot (i.e. how map was added): https://towardsdatascience.com/easy-steps-to-plot-geographic-data-on-a-map-python-11217859a2db
# BBox image taken from: http://bboxfinder.com/#53.370738,-2.312115,53.534332,-2.161372

# Manchester Map
map_img = plt.imread("https://www.eddwebster.com/downloads/manchester_map_4.png")

# Plot
fig, ax = plt.subplots(figsize=(15, 8))
ax.scatter(pd_gp_locations['lng'], pd_gp_locations['lat'], zorder=3, c=cluster_labels, cmap = cm.Dark2, edgecolor='None', alpha=0.7, s=120)
ax.scatter(centroids_pd['longitude'], centroids_pd['latitude'], zorder=2, marker='x', linewidths=2, c='k', s=50)
ax.imshow(map_img, zorder=1, extent = BBox, aspect= 'equal', alpha=0.5)
ax.set_title('GP Clusters & Centroid', fontsize = 30)
ax.set_xlabel('Longitude', fontsize = 24)
ax.set_ylabel('Latitude', fontsize = 24)
ax.set_xticks(np.arange(BBox[0],BBox[1], step=0.02))
ax.set_yticks(np.arange(BBox[2],BBox[3], step=0.025))
ax.set_xlim(BBox[0],BBox[1])
ax.set_ylim(BBox[2],BBox[3])
ax.legend([gp_scatter, centroid_scatter], ['GPs', 'GP Cluster Centroid'], loc='upper right', fontsize = 20)
display(fig)

##### K-means

In [33]:
feat_cols = ['lat', 'lng']
vec_assembler = VectorAssembler(inputCols = feat_cols, outputCol='features')
assbl = vec_assembler.transform(df_gp_locations)

# Scaling the data
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withStd=True, withMean=False)
# Compute summary statistics by fitting the StandardScaler
scalerModel = scaler.fit(assbl)

# Normalize each feature to have unit standard deviation.
assbl_scl = scalerModel.transform(assbl)
assbl_scl = assbl_scl.select('Practice_Name','features','scaledFeatures')


In [34]:
wssse = np.zeros(15)
for k in range(2,15):
    kmeans = KMeans(featuresCol='scaledFeatures',k=k)
    model = kmeans.fit(assbl_scl)
    wssse[k] = model.computeCost(assbl_scl)
    print("With K={}".format(k))
    print("Within Set Sum of Squared Errors = " + str(wssse[k]))
    print('--'*30)

/databricks/spark/python/pyspark/ml/clustering.py:349: DeprecationWarning: Deprecated in 2.4.0. It will be removed in 3.0.0. Use ClusteringEvaluator instead. You can also get the cost on the training dataset in the summary.
 DeprecationWarning)
With K=2
Within Set Sum of Squared Errors = 71.10419236733095
------------------------------------------------------------
With K=3
Within Set Sum of Squared Errors = 49.63790731371985
------------------------------------------------------------
With K=4
Within Set Sum of Squared Errors = 35.21738893853414
------------------------------------------------------------
With K=5
Within Set Sum of Squared Errors = 24.79760683928053
------------------------------------------------------------
With K=6
Within Set Sum of Squared Errors = 21.891584367763766
------------------------------------------------------------
With K=7
Within Set Sum of Squared Errors = 18.485038511298658
------------------------------------------------------------
With K=8
Within Set Sum of Squared Errors = 15.144952859326894
------------------------------------------------------------
With K=9
Within Set Sum of Squared Errors = 12.360132873596454
------------------------------------------------------------
With K=10
Within Set Sum of Squared Errors = 12.056442143899348
------------------------------------------------------------
With K=11
Within Set Sum of Squared Errors = 9.199885039328803
------------------------------------------------------------
With K=12
Within Set Sum of Squared Errors = 8.665173126546723
------------------------------------------------------------
With K=13
Within Set Sum of Squared Errors = 8.10303804676157
------------------------------------------------------------
With K=14
Within Set Sum of Squared Errors = 7.047646175972823
------------------------------------------------------------

In [35]:
fig, ax = plt.subplots(1,1, figsize =(8,6))
ax.plot(range(2,15),wssse[2:15])
ax.set_xlabel('k')
ax.set_ylabel('wssse')
display(fig)

In [36]:
# Trains a k-means model.
# Changed scaled features to features to print centroids
kmeans = KMeans(featuresCol='features', k=5, seed=62)
model = kmeans.fit(assbl_scl)

# Evaluate clustering by computing Within Set Sum of Squared Errors.
wssse = model.computeCost(assbl_scl)
print("Within Set Sum of Squared Errors = " + str(wssse))

Within Set Sum of Squared Errors = 0.03417391257975481

In [37]:
transformed = model.transform(assbl_scl).select('Practice_Name', 'prediction')
rows = transformed.collect()
df_pred = sqlContext.createDataFrame(rows)

df_km = df_gp_locations.select('Practice_Name','lat','lng')
df_km = df_km.join(df_pred, 'Practice_Name')
pd_km = df_km.toPandas()

In [38]:
# Centroids
y_lst = []
centers = model.clusterCenters()
for center in centers:
    y_lst.append(center[0])
x_lst = []
centers = model.clusterCenters()
for center in centers:
    x_lst.append(center[1])
    
y =  pd.DataFrame({'lat':y_lst})
x = pd.DataFrame({'lng':x_lst})
centroids = pd.concat([x, y], axis = 1, sort = False)
centroids

In [39]:
fig, ax = plt.subplots()
cmap = plt.cm.get_cmap('jet')
for i, cluster in pd_km.groupby('prediction'):
    _ = ax.scatter(cluster['lng'], cluster['lat'], label=i)
centroid_scatter = ax.scatter(centroids['lng'], centroids['lat'], marker='x', linewidths=2, c='k', s=50)


ax.set_title('K-Means Clustering')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.legend()
ax.grid(True)

display()

In [40]:
output = transformed.join(df_gp_locations, 'Practice_Name', how='inner')
display(output)

Practice_Name,prediction,Locality,Neighbourhood,Adress1,Address2,Address3,Address4,Postcode,lat,lng
The Robert Darbishire Practice,3,Central,"Hulme, Moss Side & Rusholme",Rusholme Health Centre,Walmer Street,Rusholme,Manchester,M14 5NP,53.45461799017,-2.226163245935
David Medical Centre,4,South,"Didsbury, Burnage & Chorlton",274 Barlow Moor Road,Chorlton-Cum-Hardy,Manchester,null,M21 8HA,53.431842891301,-2.268651654184
Corkland Road Medical Practice,4,Central,"Chorlton, Whalley Range & Fallowfield",Corkland Road Medical Practice,9 Corkland Road,Chorlton-Cum-Hardy,Manchester,M21 8UP,53.441880257389,-2.273848643025
Ailsa Craig Medical Practice,3,Central,Ardwick & Longsight,Ailsa Craig Medical Practice,270 Dickenson Road,Longsight,Manchester,M13 0YL,53.454182682053,-2.208541995009
Dickenson Road Medical Centre,3,Central,Ardwick & Longsight,Dickenson Road Medical Centre,357-359 Dickenson Road,Longsight,Manchester,M13 0WQ,53.455885324192,-2.201080826099
Victoria Mill Medical Practice,2,North,"Miles Platting, Newton Heath, City Centre & Moston",Victoria Mill Health Care Centre,10 Lower Vickers Street,Miles Platting,Manchester,M40 7LH,53.489631997573,-2.21429250676
Wellfield Medical Centre,2,North,Crumpsall & Cheetham,Wellfield Medical Centre,53-55 Crescent Road,Crumpsall,Manchester,M8 9JT,53.512544477139,-2.242003895599
Chorlton Family Practice,4,Central,"Chorlton, Whalley Range & Fallowfield",Chorlton Family Practice,1 Nicolas Road,Chorlton Cum Hardy,Manchester,M21 9NJ,53.443781203356,-2.279717375134
Charlestown Surgery,0,North,"Higher Blackley, Harphurhey & Charlestown",Charlestown Medical Practice,Charlestown Road,Blackley,Manchester,M9 7ED,53.522629351066,-2.187022900092
Collegiate Medical Centre,2,North,Crumpsall & Cheetham,Collegiate Medical Centre,407 Cheetham Hill Road,Manchester,Lancashire,M8 0DA,53.504457746024,-2.236092934522


In [41]:
output = transformed.join(df_gp_locations, 'Practice_Name', how='inner')
pdf = output.toPandas()

colors = {0 : 'red', 1 : 'blue', 2 : 'green', 3 : 'yellow', 4 : 'purple'}

m = folium.Map(location=[53.485506, -2.246124], zoom_start=11) # create a base map for Mancheste

pdf.apply(lambda row:folium.CircleMarker(location=[row['lat'], row['lng']], 
                                              radius=10, fill_color=colors[row['prediction']])
                                             .add_to(m), axis=1)

# this base maps makes the markers easier to read, but you lose the points of interest (ie. Doctors offices)
folium.TileLayer('cartodbpositron').add_to(m)

for point in range(0, len(centroids)):
  folium.Marker(centroids[point]).add_to(map)

Out[34]: <folium.raster_layers.TileLayer at 0x7efd44215c50>

In [42]:
# display the map
m

Make this Notebook Trusted to load map: File -> Trust Notebook